In [73]:
# FETCH HISTORICAL DATA

from torchtrade.data.cdd import CryptoDataDownload
import pandas as pd

cdd = CryptoDataDownload()

bitstamp_data = pd.concat([
    cdd.fetch("Bitstamp", "USD", "BTC", "1h").add_prefix("BTC:"),
], axis=1)

# bitstamp_data.drop("BTC:volume", axis=1, inplace=True)
bitstamp_data

,BTC:date,BTC:unix,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume
0,2018-05-15 06:00:00,1526364000,8733.86,8796.68,8707.28,8740.99,5.599300e+02
1,2018-05-15 07:00:00,1526367600,8740.99,8766.00,8721.11,8739.00,2.735800e+02
2,2018-05-15 08:00:00,1526371200,8739.00,8750.27,8660.53,8728.49,9.177900e+02
3,2018-05-15 09:00:00,1526374800,8728.49,8754.40,8701.35,8708.32,1.826200e+02
4,2018-05-15 10:00:00,1526378400,8708.32,8865.00,8695.11,8795.90,1.260690e+03
...,...,...,...,...,...,...,...
57686,2024-12-12 20:00:00,1734033600,99472.00,100408.00,99472.00,100032.00,8.690325e+06
57687,2024-12-12 21:00:00,1734037200,100036.00,100146.00,99733.00,99884.00,4.903087e+06
57688,2024-12-12 22:00:00,1734040800,99798.00,100498.00,99553.00,100497.00,6.127996e+06
57689,2024-12-12 23:00:00,1734044400,100496.00,100496.00,99916.00,100068.00,3.106767e+06


In [81]:
# DEFINE EXCHANGES

from torchtrade.oms.exchanges import Exchange
from torchtrade.feed.core.base import Stream
from torchtrade.oms.services.execution.simulated import execute_order


bitstamp = Exchange("bitstamp", service=execute_order)(
    Stream.source(list(bitstamp_data['BTC:close']), dtype="float").rename("USD-BTC")
)

In [82]:
# DEFINE ADDITIONAL DATA (TA, etc)
import ta
from torchtrade.feed.core import Stream, DataFeed, NameSpace

bitstamp_btc = bitstamp_data.loc[:, bitstamp_data.columns.str.startswith("BTC")].copy()

ta.add_all_ta_features(
    bitstamp_btc,
    colprefix='BTC:',
    **{k: "BTC:" + k for k in ['open', 'high', 'low', 'close', 'volume']}
)

with NameSpace("bitstamp"):
    bitstamp_steams = [
        Stream.source(list(bitstamp_btc[c]), dtype="float").rename(c) for c in bitstamp_btc.columns
    ]
    
feed = DataFeed(bitstamp_steams)

feed.next()

{'bitstamp:/BTC:date': Timestamp('2018-05-15 06:00:00'),
 'bitstamp:/BTC:unix': 1526364000,
 'bitstamp:/BTC:open': 8733.86,
 'bitstamp:/BTC:high': 8796.68,
 'bitstamp:/BTC:low': 8707.28,
 'bitstamp:/BTC:close': 8740.99,
 'bitstamp:/BTC:volume': 559.93,
 'bitstamp:/BTC:volume_adi': -137.6651163311054,
 'bitstamp:/BTC:volume_obv': 559.93,
 'bitstamp:/BTC:volume_cmf': nan,
 'bitstamp:/BTC:volume_fi': nan,
 'bitstamp:/BTC:volume_em': nan,
 'bitstamp:/BTC:volume_sma_em': nan,
 'bitstamp:/BTC:volume_vpt': nan,
 'bitstamp:/BTC:volume_vwap': nan,
 'bitstamp:/BTC:volume_mfi': nan,
 'bitstamp:/BTC:volume_nvi': 1000.0,
 'bitstamp:/BTC:volatility_bbm': nan,
 'bitstamp:/BTC:volatility_bbh': nan,
 'bitstamp:/BTC:volatility_bbl': nan,
 'bitstamp:/BTC:volatility_bbw': nan,
 'bitstamp:/BTC:volatility_bbp': nan,
 'bitstamp:/BTC:volatility_bbhi': 0.0,
 'bitstamp:/BTC:volatility_bbli': 0.0,
 'bitstamp:/BTC:volatility_kcc': nan,
 'bitstamp:/BTC:volatility_kch': 8837.716666666667,
 'bitstamp:/BTC:volatility

In [88]:
# SET PORTFOLIO
from torchtrade.oms.wallets.wallet import Wallet
from torchtrade.oms.wallets.portfolio import Portfolio

from torchtrade.oms.instruments import USD, BTC, ETH, LTC

portfolio = Portfolio(
    base_instrument=USD, 
    wallets=[
        Wallet(bitstamp, 1 * BTC),
        # Wallet(bitstamp, 1 * BTC),
        # ...
    ]
)

In [90]:
# ENVIRONMENT
import torchtrade.env.default as default

env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="simple",
    feed=feed,
    window_size=15,
    enable_logger=True
)

env.observer.feed.next()

AttributeError: 'IterableStream' object has no attribute 'mul'